In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#sayali
#Imports
import pandas as pd
from datetime import datetime

#Variables
cabin = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Cabin_Info.csv")
schedule = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Class_Schedule.csv")
attendance = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Employee_Attendance_Data.csv")
leave = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Leave_Info.csv")
slots = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Slots.csv")
calender = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Academic_Calender.csv")

attendance['Date'] = pd.to_datetime(attendance['Date'], format="%d-%m-%Y")
calender['Date'] = pd.to_datetime(calender['Date'], format="%d-%m-%Y")

#Input
empID = int(input("Enter Employee ID: "))
start_date = pd.to_datetime(input("Enter start date (DD-MM-YYYY): "), format="%d-%m-%Y")
end_date = pd.to_datetime(input("Enter end date (DD-MM-YYYY): "), format="%d-%m-%Y")

#Functions
def get_schedule(emp_id, schedule_df):
    return schedule_df[schedule_df['Employee ID'] == emp_id][['Class Building', 'Slot ID']].values.tolist()

def build_class_times_by_date(emp_schedule, slots_df, calender_df):
    date_to_times = {}
    instructional = calender_df[calender_df['Status'] == 'Instructional']
    for _, slot_id in emp_schedule:
        slot_rows = slots_df[slots_df['Slot_ID'] == slot_id]
        for _, slot in slot_rows.iterrows():
            day = slot['Day']
            start_time = datetime.strptime(slot['Start_Time'], "%H:%M").time()
            end_time = datetime.strptime(slot['End_Time'], "%H:%M").time()
            matching_dates = instructional[instructional['Day'] == day]
            for _, date_row in matching_dates.iterrows():
                date = date_row['Date']
                if date not in date_to_times:
                    date_to_times[date] = (start_time, end_time)
                else:
                    existing_start, existing_end = date_to_times[date]
                    date_to_times[date] = (min(existing_start, start_time), max(existing_end, end_time))
    return date_to_times

#Processing
date_range_df = calender[
    (calender['Date'] >= start_date) &
    (calender['Date'] <= end_date)
]

emp_schedule = get_schedule(empID, schedule)
class_times_by_date = build_class_times_by_date(emp_schedule, slots, calender)

# Attendance Report
print("\nAttendance Report:")
for _, row in date_range_df.iterrows():
    date = row['Date']
    day = row['Day']
    status = row['Status']
    date_str = date.strftime('%d-%m-%Y')
    label = f"{date_str} ({day})"
    if status != 'Instructional':
        print(f"{label} - Non Instructional Day")
        continue
    class_start, class_end = class_times_by_date.get(date, (None, None))
    emp_attendance = attendance[
        (attendance['Employee ID'] == empID) &
        (attendance['Date'] == date)
    ]
    if emp_attendance.empty:
        entry_status = "Absent"
        exit_status = "Absent"
    else:
        punch_in = datetime.strptime(emp_attendance.iloc[0]['Punch In Time'], "%H:%M").time()
        punch_out = datetime.strptime(emp_attendance.iloc[0]['Punch Out Time'], "%H:%M").time()
        if class_start:
            entry_status = "Arrived In Time" if punch_in <= class_start else "Late Entry"
        else:
            entry_status = "Arrived (No Class)"
        if class_end:
            exit_status = "Left On Time" if punch_out >= class_end else "Left Early"
        else:
            exit_status = "Left (No Class)"
    print(f"{label} - Entry: {entry_status} | Exit: {exit_status}")

Enter Employee ID: 13665
Enter start date (DD-MM-YYYY): 07-04-2025
Enter end date (DD-MM-YYYY): 13-04-2025

Attendance Report:
07-04-2025 (Monday) - Entry: Arrived (No Class) | Exit: Left (No Class)
08-04-2025 (Tuesday) - Entry: Arrived (No Class) | Exit: Left (No Class)
09-04-2025 (Wednesday) - Entry: Arrived (No Class) | Exit: Left (No Class)
10-04-2025 (Thursday) - Entry: Arrived In Time | Exit: Left On Time
11-04-2025 (Friday) - Entry: Arrived (No Class) | Exit: Left (No Class)
12-04-2025 (Saturday) - Entry: Arrived (No Class) | Exit: Left (No Class)
13-04-2025 (Sunday) - Non Instructional Day


In [ ]:
#1 to give attendence details of an employee for a particular day
import pandas as pd
from datetime import datetime

# Load the attendance file
attendance = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Employee_Attendance_Data.csv")

# Convert Date column to datetime
attendance['Date'] = pd.to_datetime(attendance['Date'], format="%d-%m-%Y")

# Input from user
emp_id = int(input("Enter Employee ID: "))
date_input = input("Enter Date (DD-MM-YYYY): ")
query_date = pd.to_datetime(date_input, format="%d-%m-%Y")

# Filter the attendance
record = attendance[
    (attendance['Employee ID'] == emp_id) &
    (attendance['Date'] == query_date)
]

# Output result
if record.empty:
    print(f"No attendance record found for Employee ID {emp_id} on {date_input}")
else:
    row = record.iloc[0]
    print(f"\nAttendance for Employee ID {emp_id} on {date_input}:")
    print(f"Punch In Time     : {row['Punch In Time']} ({row['Punch In Building']}, Device: {row['Punch In Device']})")
    print(f"Punch Out Time    : {row['Punch Out Time']} ({row['Punch Out Building']}, Device: {row['Punch Out Device']})")


Enter Employee ID: 13665
Enter Date (DD-MM-YYYY): 09-04-2025

Attendance for Employee ID 13665 on 09-04-2025:
Punch In Time     : 8:00 (GDN, Device: GDN1)
Punch Out Time    : 18:00 (ALM, Device: ALM2)


In [35]:
#2 if cabin is in different building and class in different building but punching elsewhere
import pandas as pd
from datetime import datetime

# Load data
attendance = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Employee_Attendance_Data.csv")
cabin = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Cabin_Info.csv")
schedule = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Class_Schedule.csv")
slots = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Slots.csv")
calendar = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Academic_Calender.csv")

# Preprocess
attendance['Date'] = pd.to_datetime(attendance['Date'], format="%d-%m-%Y")
calendar['Date'] = pd.to_datetime(calendar['Date'], format="%d-%m-%Y")

# Input
emp_id = int(input("Enter Employee ID: "))
date_input = input("Enter Date (DD-MM-YYYY): ")
query_date = pd.to_datetime(date_input, format="%d-%m-%Y")

# Get day name
day_row = calendar[calendar['Date'] == query_date]
if day_row.empty:
    print("Invalid date entered (not in academic calendar).")
    exit()

day_name = day_row['Day'].values[0]

# Get punch in data
record = attendance[
    (attendance['Employee ID'] == emp_id) &
    (attendance['Date'] == query_date)
]

if record.empty:
    print(f"No attendance found for Employee ID {emp_id} on {date_input}")
else:
    row = record.iloc[0]
    punch_in_building = row['Punch In Building']
    punch_in_device = row['Punch In Device']
    punch_in_time = row['Punch In Time']

    punch_out_building = row['Punch Out Building']
    punch_out_device = row['Punch Out Device']
    punch_out_time = row['Punch Out Time']

    # Get cabin building
    emp_cabin = cabin[cabin['Emp_ID'] == emp_id]
    cabin_building = emp_cabin['Building'].values[0] if not emp_cabin.empty else None

    # Get class buildings for that day
    emp_schedule = schedule[schedule['Employee ID'] == emp_id]
    relevant_slots = slots[slots['Day'] == day_name]
    class_buildings = emp_schedule[
        emp_schedule['Slot ID'].isin(relevant_slots['Slot_ID'])
    ]['Class Building'].unique().tolist()

    # Prepare categorized allowed buildings
    allowed_class_buildings = set(class_buildings)
    allowed_cabin_building = set([cabin_building]) if cabin_building else set()
    total_allowed_buildings = allowed_class_buildings.union(allowed_cabin_building)

    # Output
    print(f"\nAttendance for Employee ID {emp_id} on {date_input}")
    print(f"Punch In Time      : {punch_in_time}")
    print(f"Punch In Building  : {punch_in_building}")
    print(f"Punch In Device    : {punch_in_device}")
    print(f"Punch Out Time     : {punch_out_time}")
    print(f"Punch Out Building : {punch_out_building}")
    print(f"Punch Out Device   : {punch_out_device}")
    print(f"\nAllowed Buildings:")
    if allowed_class_buildings:
        print(f"Class Buildings    : {', '.join(allowed_class_buildings)}")
    if allowed_cabin_building:
        print(f"Cabin Building     : {', '.join(allowed_cabin_building)}")

    # Suspicious detection
    if punch_in_building not in total_allowed_buildings:
        print(f"\nSuspicious Punch In Detected!")
        print(f"Employee punched in from '{punch_in_building}' using device '{punch_in_device}', which is not an allowed location.")

    if punch_out_building not in total_allowed_buildings:
        print(f"\nSuspicious Punch Out Detected!")
        print(f"Employee punched out from '{punch_out_building}' using device '{punch_out_device}', which is not an allowed location.")

    if punch_in_building in total_allowed_buildings and punch_out_building in total_allowed_buildings:
        print("\nBoth punch locations are valid.")


Enter Employee ID: 13143
Enter Date (DD-MM-YYYY): 08-04-2025

Attendance for Employee ID 13143 on 08-04-2025
Punch In Time      : 8:00
Punch In Building  : PRP
Punch In Device    : PRP1
Punch Out Time     : 18:00
Punch Out Building : GB
Punch Out Device   : GB2

Allowed Buildings:
Cabin Building     : PRP

Suspicious Punch Out Detected!
Employee punched out from 'GB' using device 'GB2', which is not an allowed location.


In [12]:
#3 attendence report with building info
# Imports
import pandas as pd
from datetime import datetime

# Load CSVs
cabin = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Cabin_Info.csv")
schedule = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Class_Schedule.csv")
attendance = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Employee_Attendance_Data.csv")
leave = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Leave_Info.csv")
slots = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Slots.csv")
calender = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Academic_Calender.csv")

# Preprocessing
attendance['Date'] = pd.to_datetime(attendance['Date'], format="%d-%m-%Y")
calender['Date'] = pd.to_datetime(calender['Date'], format="%d-%m-%Y")

# Input
empID = int(input("Enter Employee ID: "))
start_date = pd.to_datetime(input("Enter start date (DD-MM-YYYY): "), format="%d-%m-%Y")
end_date = pd.to_datetime(input("Enter end date (DD-MM-YYYY): "), format="%d-%m-%Y")

# Functions
def get_schedule(emp_id, schedule_df):
    return schedule_df[schedule_df['Employee ID'] == emp_id][['Class Building', 'Slot ID']].values.tolist()

def build_class_times_by_date(emp_schedule, slots_df, calender_df):
    date_to_times = {}
    instructional = calender_df[calender_df['Status'] == 'Instructional']
    for _, slot_id in emp_schedule:
        slot_rows = slots_df[slots_df['Slot_ID'] == slot_id]
        for _, slot in slot_rows.iterrows():
            day = slot['Day']
            start_time = datetime.strptime(slot['Start_Time'], "%H:%M").time()
            end_time = datetime.strptime(slot['End_Time'], "%H:%M").time()
            matching_dates = instructional[instructional['Day'] == day]
            for _, date_row in matching_dates.iterrows():
                date = date_row['Date']
                if date not in date_to_times:
                    date_to_times[date] = (start_time, end_time)
                else:
                    existing_start, existing_end = date_to_times[date]
                    date_to_times[date] = (min(existing_start, start_time), max(existing_end, end_time))
    return date_to_times

# Processing
date_range_df = calender[
    (calender['Date'] >= start_date) &
    (calender['Date'] <= end_date)
]

emp_schedule = get_schedule(empID, schedule)
class_times_by_date = build_class_times_by_date(emp_schedule, slots, calender)

# Attendance Report
print("\nAttendance Report:")
for _, row in date_range_df.iterrows():
    date = row['Date']
    day = row['Day']
    status = row['Status']
    date_str = date.strftime('%d-%m-%Y')
    label = f"{date_str} ({day})"

    if status != 'Instructional':
        print(f"{label} - Non Instructional Day")
        continue

    class_start, class_end = class_times_by_date.get(date, (None, None))
    emp_attendance = attendance[
        (attendance['Employee ID'] == empID) &
        (attendance['Date'] == date)
    ]

    if emp_attendance.empty:
        entry_status = "Absent"
        exit_status = "Absent"
        punch_in_info = "N/A"
        punch_out_info = "N/A"
    else:
        row = emp_attendance.iloc[0]
        punch_in_time = datetime.strptime(row['Punch In Time'], "%H:%M").time()
        punch_out_time = datetime.strptime(row['Punch Out Time'], "%H:%M").time()
        punch_in_building = row['Punch In Building']
        punch_out_building = row['Punch Out Building']
        punch_in_device = row['Punch In Device']
        punch_out_device = row['Punch Out Device']

        punch_in_info = f"{row['Punch In Time']} ({punch_in_building}, {punch_in_device})"
        punch_out_info = f"{row['Punch Out Time']} ({punch_out_building}, {punch_out_device})"

        if class_start:
            entry_status = "Arrived In Time" if punch_in_time <= class_start else "Late Entry"
        else:
            entry_status = "Arrived (No Class)"

        if class_end:
            exit_status = "Left On Time" if punch_out_time >= class_end else "Left Early"
        else:
            exit_status = "Left (No Class)"

    print(f"{label} - Entry: {entry_status} at {punch_in_info} | Exit: {exit_status} at {punch_out_info}")


Enter Employee ID: 15701
Enter start date (DD-MM-YYYY): 07-04-2025
Enter end date (DD-MM-YYYY): 13-04-2025

Attendance Report:
07-04-2025 (Monday) - Entry: Arrived (No Class) at 8:30 (GDN, GDN1) | Exit: Left (No Class) at 18:00 (GDN, GDN2)
08-04-2025 (Tuesday) - Entry: Late Entry at 8:10 (MGR, MGR1) | Exit: Left On Time at 18:00 (ALM, ALM2)
09-04-2025 (Wednesday) - Entry: Arrived (No Class) at 8:00 (MGR, MGR1) | Exit: Left (No Class) at 18:00 (ALM, ALM2)
10-04-2025 (Thursday) - Entry: Arrived (No Class) at 8:00 (MGR, MGR1) | Exit: Left (No Class) at 18:00 (PRP, PRP2)
11-04-2025 (Friday) - Entry: Arrived (No Class) at 7:55 (GDN, GDN1) | Exit: Left (No Class) at 18:20 (ALM, ALM2)
12-04-2025 (Saturday) - Entry: Arrived (No Class) at 8:20 (GDN, GDN1) | Exit: Left (No Class) at 18:00 (ALM, ALM2)
13-04-2025 (Sunday) - Non Instructional Day


In [7]:
#4 if you are late for class or on time for both you will get punch in details
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Load data
attendance = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Employee_Attendance_Data.csv")
schedule = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Class_Schedule.csv")
slots = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Slots.csv")
calendar = pd.read_csv("/content/drive/MyDrive/python_project/attendance_analysis_system/Academic_Calender.csv")

# Convert date formats
attendance['Date'] = pd.to_datetime(attendance['Date'], format="%d-%m-%Y")
calendar['Date'] = pd.to_datetime(calendar['Date'], format="%d-%m-%Y")

# Inputs
emp_id = int(input("Enter Employee ID: "))
date_input = input("Enter Date (DD-MM-YYYY): ")
query_date = pd.to_datetime(date_input, format="%d-%m-%Y")

# Get Day Name
calendar_day = calendar[calendar['Date'] == query_date]
if calendar_day.empty or calendar_day.iloc[0]['Status'] != 'Instructional':
    print("This is not an instructional day.")
    exit()

day_name = calendar_day.iloc[0]['Day']

# Get attendance
record = attendance[
    (attendance['Employee ID'] == emp_id) &
    (attendance['Date'] == query_date)
]

if record.empty:
    print("No attendance record found.")
    exit()

row = record.iloc[0]
punch_in_time_str = row['Punch In Time']
punch_in_time = datetime.strptime(punch_in_time_str, "%H:%M").time()
punch_in_building = row['Punch In Building']
punch_in_device = row['Punch In Device']

# Get class start time
emp_schedule = schedule[schedule['Employee ID'] == emp_id]
day_slots = slots[slots['Day'] == day_name]

# Filter slots for that day
class_slots = emp_schedule[emp_schedule['Slot ID'].isin(day_slots['Slot_ID'])]

# Output block
print(f"\nAttendance Check for Employee ID {emp_id} on {date_input}")
print(f"Punch In Time      : {punch_in_time_str}")
print(f"Punch In Building  : {punch_in_building}")
print(f"Punch In Device    : {punch_in_device}")

if class_slots.empty:
    print("No scheduled classes for this day.")
    print("Note: Cannot check for late entry in class.\n")
else:
    # Get earliest class start time
    min_time = None
    for slot_id in class_slots['Slot ID']:
        times = day_slots[day_slots['Slot_ID'] == slot_id]['Start_Time']
        for t in times:
            slot_start = datetime.strptime(t, "%H:%M").time()
            if min_time is None or slot_start < min_time:
                min_time = slot_start

    print(f"Class Start Time   : {min_time}")

    if punch_in_time <= min_time:
        print("Arrived On Time")
    else:
        print("Late Entry for Class!!!")


Enter Employee ID: 15701
Enter Date (DD-MM-YYYY): 07-04-2025

Attendance Check for Employee ID 15701 on 07-04-2025
Punch In Time      : 8:30
Punch In Building  : GDN
Punch In Device    : GDN1
No scheduled classes for this day.
Note: Cannot check for late entry in class.

